# Projeto de Extração de Dados de Engenharia de Dados da ADA Tech

## Teste da chamada do arquivo Silver

In [33]:
import pandas as pd

df = pd.read_parquet("data/silver/silver.parquet")

df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 304 entries, 0 to 32
Data columns (total 13 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   author       304 non-null    object
 1   content      304 non-null    object
 2   description  304 non-null    object
 3   publishedAt  304 non-null    object
 4   title        285 non-null    object
 5   url          304 non-null    object
 6   urlToImage   300 non-null    object
 7   load_date    304 non-null    object
 8   year         304 non-null    int32 
 9   month        304 non-null    int32 
 10  day          304 non-null    int32 
 11  source_id    45 non-null     object
 12  source_name  304 non-null    object
dtypes: int32(3), object(10)
memory usage: 29.7+ KB


In [34]:
import openpyxl
df.to_excel("texxxxxt.xlsx")

In [35]:
df.head(2)

,author,content,description,publishedAt,title,url,urlToImage,load_date,year,month,day,source_id,source_name
0,Agência Brasil,Texto: Solimar Luz/Rádio Nacional/Agência Bras...,Produto não é tóxico para meio ambiente nem pa...,2024-03-25,Cientistas desenvolvem armadilha para combater...,https://gizmodo.uol.com.br/cientistas-desenvol...,https://gizmodo.uol.com.br/wp-content/blogs.di...,2024-04-09,2024,3,25,None,Uol.com.br
1,Nayra Teles,Os casos de dengue no Brasil já ultrapassam 2 ...,Uma pesquisa brasileira testou a efetividade d...,2024-03-28,Água sanitária mata o mosquito da dengue?,https://olhardigital.com.br/2024/03/28/medicin...,https://olhardigital.com.br/wp-content/uploads...,2024-04-09,2024,3,28,None,Olhardigital.com.br


## 4. Dados transformados para consulta do público final







#### 4.1 - Quantidade de notícias por ano, mês e dia de publicação;

In [36]:
quantidade_por_data = df.groupby(['day', 'month', 'year']).size().reset_index().rename(columns={0: 'quantidade'})
quantidade_por_data

,day,month,year,quantidade
0,1,4,2024,31
1,2,4,2024,34
2,3,4,2024,33
3,4,4,2024,29
4,5,4,2024,27
5,6,4,2024,17
6,25,3,2024,18
7,26,3,2024,26
8,27,3,2024,43
9,28,3,2024,46


In [37]:
quantidade_por_mes = df.groupby(df['month']).size().reset_index().rename(columns={0: 'quantidade'})
quantidade_por_mes

,month,quantidade
0,3,133
1,4,171


In [38]:
quantidade_por_ano = df.groupby(df['year']).size().reset_index().rename(columns={0: 'quantidade'})
quantidade_por_ano

,year,quantidade
0,2024,304


#### 4.2 - Quantidade de notícias por fonte e autor;

In [39]:
qtde_noticias_fonte_autor = df.groupby(df['source_name']).size().sort_values(ascending=False).reset_index().rename(columns={0: 'quantidade'})
qtde_noticias_fonte_autor.head(10)

,source_name,quantidade
0,Metropoles.com,63
1,R7.com,56
2,Globo,41
3,Terra.com.br,33
4,Ig.com.br,31
5,Uol.com.br,24
6,Jornaldebrasilia.com.br,11
7,Olhardigital.com.br,8
8,Abril.com.br,4
9,InfoMoney,3


In [55]:
df_agg_source_author = pd.read_parquet('data/gold/aggregateBySourceAuthor.parquet')
df_agg_source_author.sort_values(by="number_articles", ascending=False).reset_index(drop="index")

,source_name,author,number_articles
0,Globo,Não Informado,41
1,R7.com,Do R7,33
2,Jornaldebrasilia.com.br,Jornal de Brasília,11
3,Ig.com.br,"Agência Brasil, Agência Brasil",9
4,Uol.com.br,Não Informado,8
...,...,...,...
109,Metropoles.com,ONU News,1
110,Metropoles.com,Mateus Salomão,1
111,Metropoles.com,Luciano Arcoverde,1
112,Metropoles.com,Luana Viana,1


#### 4.3 - Quantidade de aparições de 3 palavras chaves por ano, mês e dia de publicação (as 3 palavras chaves serão as mesmas usadas para fazer os filtros de relevância do item 2 (2. Definir Critérios de Relevância)).

In [40]:
from modules.etl import contagem_palavras

In [41]:
df_qtde = contagem_palavras()
df_qtde.head()

,Palavra,Quantidade
0,dengue,90
1,casos,63
2,Saúde,45
3,doença,34
4,contra,33


In [42]:
contagem = contagem_palavras()

# Obter as top 3 palavras mais comuns
top3_palavras = contagem.head(3)

# Criar uma nova coluna para cada uma das top 3 palavras e contar as ocorrências
for palavra in top3_palavras['Palavra']:
    df[palavra] = df['description'].str.lower().str.count(palavra.lower())

# Agrupar os resultados pela data de publicação e somar o número de ocorrências das top 3 palavras em cada grupo
for palavra in top3_palavras['Palavra']:
    quantidade_por_dia = df.groupby(['year', 'month', 'day', palavra]).size().reset_index(name='contagem')
    quantidade_por_dia = quantidade_por_dia[quantidade_por_dia[palavra] != 0]
    print(f'Quantidade de ocorrências da palavra "{palavra}" por ano, mês e dia:')
    print(quantidade_por_dia)


Quantidade de ocorrências da palavra "dengue" por ano, mês e dia:
    year  month  day  dengue  contagem
1   2024      3   25       1         5
3   2024      3   26       1        12
4   2024      3   26       2         1
6   2024      3   27       1        17
8   2024      3   28       1        23
9   2024      3   28       2         1
11  2024      4    1       1        15
12  2024      4    1       2         1
14  2024      4    2       1        14
15  2024      4    2       2         1
17  2024      4    3       1         9
19  2024      4    4       1        11
21  2024      4    5       1         9
22  2024      4    5       2         1
24  2024      4    6       1         5
Quantidade de ocorrências da palavra "casos" por ano, mês e dia:
    year  month  day  casos  contagem
1   2024      3   25      1         7
3   2024      3   26      1         7
5   2024      3   27      1         6
6   2024      3   27      2         1
8   2024      3   28      1        12
9   2024      3  

In [43]:
contagem = contagem_palavras()

# Obter as top 3 palavras mais comuns
top3_palavras = contagem.head(3)

# Criar uma nova coluna para cada uma das top 3 palavras
for palavra in top3_palavras['Palavra']:
    df[palavra] = df['description'].str.lower().str.contains(palavra.lower())

# Criar uma coluna para verificar se todas as top 3 palavras estão presentes em cada notícia
df['todas_top3'] = df[top3_palavras['Palavra']].all(axis=1)

# Agrupar os resultados pela data de publicação e contar quantas notícias têm as top 3 palavras em cada grupo
quantidade_por_dia = df.groupby(['year', 'month', 'day'])['todas_top3'].sum().reset_index()

print("Quantidade de notícias que contêm as top 3 palavras por ano, mês e dia:")
quantidade_por_dia


Quantidade de notícias que contêm as top 3 palavras por ano, mês e dia:


,year,month,day,todas_top3
0,2024,3,25,1
1,2024,3,26,0
2,2024,3,27,1
3,2024,3,28,2
4,2024,4,1,1
5,2024,4,2,1
6,2024,4,3,2
7,2024,4,4,0
8,2024,4,5,1
9,2024,4,6,0
